In [1]:
import numpy as np
import matplotlib.pyplot as plt
from decimal import Decimal # used to round the numbers so they look pretty

Now we consider the case where a flock may have multiple species of birds. We suppose there are `num_flocks` flocks and `num_species` species.

In [2]:
# set number of species and flocks
num_flocks = 2
num_species = 3


In [3]:
# set initial conditions 

# the following convention will be used: first dimension will represent which flock, second is which species, third is the compartment.
init_val = np.zeros((num_flocks, num_species, 6)) # Six possible compartment: S, E_S, E_A, I_S, I_A, R

# first let all birds to start susceptible, also choose population size here.
init_val[:,:,0] += 100 # 100 birds per flock per species, can be changed

# then choose a bird to be exposed (symptomatic), here we assume it to be the first species in the first flock.
init_val[0,0,0] -= 1
init_val[0,0,1] += 1

init_val

array([[[ 99.,   1.,   0.,   0.,   0.,   0.],
        [100.,   0.,   0.,   0.,   0.,   0.],
        [100.,   0.,   0.,   0.,   0.,   0.]],

       [[100.,   0.,   0.,   0.,   0.,   0.],
        [100.,   0.,   0.,   0.,   0.,   0.],
        [100.,   0.,   0.,   0.,   0.,   0.]]])

In [4]:
# this is the maximum number of events that would occur, typically the number will not be reached, but 
# for diseases that does not die out this is necessary to not fall into an infinite while loop.
max_events = 500000

# set the time and state sequence
t = [0] + [None] * max_events
y = [init_val] + [None] * max_events


We consider each of the parameter: firstly the infection rate (from symptomatic birds). The $(a,b,c,d)$-th entry of the 4-rank tensor `beta_S` means "the infection rate from a symptomatic infectious bird in flock $a$ species $b$, to flock $c$ species $d$".

In [5]:
# initialise the infection rate tensor
beta_S = np.zeros((num_flocks, num_species, num_flocks, num_species))

# insert some values, can be changed
for a in range(num_flocks):
    for b in range(num_species):
        for c in range(num_flocks):
            for d in range(num_species):
                if a == c and b == d:
                    beta_S[a,b,c,d] = 4
                elif a == c and b != d:
                    beta_S[a,b,c,d] = 3.5
                elif a != c and b == d:
                    beta_S[a,b,c,d] = 1.5
                else:
                    beta_S[a,b,c,d] = 1.2


Same idea applies to asymptomatic birds.

In [6]:
# initialise the infection rate tensor
beta_A = np.zeros((num_flocks, num_species, num_flocks, num_species))

# insert some values, can be changed
for a in range(num_flocks):
    for b in range(num_species):
        for c in range(num_flocks):
            for d in range(num_species):
                if a == c and b == d:
                    beta_A[a,b,c,d] = 2
                elif a == c and b != d:
                    beta_A[a,b,c,d] = 1.8
                elif a != c and b == d:
                    beta_A[a,b,c,d] = 1
                else:
                    beta_A[a,b,c,d] = 0.8


Now consider the latency and infectious period, probability of symptom display.

In [10]:
# latency and infectious period
latency_period_S = 1 * np.ones(num_species)
sigma_S = np.ones(num_species) / latency_period_S
latency_period_A = 0.5 * np.ones(num_species)
sigma_A = np.ones(num_species) / latency_period_A

infectious_period_S = 4 * np.ones(num_species)
gamma_S = np.ones(num_species) / infectious_period_S
infectious_period_A = 2 * np.ones(num_species)
gamma_A = np.ones(num_species) / infectious_period_A

# probability of displaying symptoms.
p_S = np.array([0.8] * num_species)
p_A = np.ones(num_species) - p_S

Gillespie... to be continued